In [8]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [5]:
X, y = make_classification(n_samples=500, n_features=5, random_state=42)
y = np.where(y == 0, -1, 1)  # Convert labels to {-1, 1}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
class AdaBoost:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.alphas = []
        self.models = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = np.ones(n_samples) / n_samples
        for _ in range(self.n_estimators):
            tree = DecisionTreeClassifier(max_depth=1)
            tree.fit(X, y, sample_weight=w)
            y_pred = tree.predict(X)
            err = np.sum(w * (y_pred != y)) / np.sum(w)
            alpha = 0.5 * np.log((1 - err) / (err + 1e-10))
            self.alphas.append(alpha)
            self.models.append(tree)
            w = w * np.exp(-alpha * y * y_pred)
            w /= np.sum(w)

    def predict(self, X):
        pred = sum(alpha * model.predict(X) for alpha, model in zip(self.alphas, self.models))
        return np.sign(pred)

In [9]:
adaboost = AdaBoost(n_estimators=50)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.93
